In [ ]:
!pip install CoolProp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 56.9 MB/s eta 0:00:00


In [ ]:
import CoolProp.CoolProp as CP

# Get fluid properties from CoolProp
fluid = input("Enter the fluid type (e.g. water, air): ")
T_inf = float(input("Enter the fluid temperature [°C]: "))
T_s = float(input("Enter the surface temperature [°C]: "))
p_inf = float(input("Enter the fluid pressure [bar]: "))
#Dependendo da correlação que será utilizada, as propriedades dos fluidos derão ser avaliadas na temperatura média do fuido, Tf =(T_inf+T_s)/2
T_f =(T_inf+T_s)/2
rho = CP.PropsSI("D", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
k = CP.PropsSI("conductivity", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
mu = CP.PropsSI("viscosity", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
nu = mu/rho
Pr = CP.PropsSI("Prandtl", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
cp = CP.PropsSI("CPMASS", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
beta = CP.PropsSI("ISOBARIC_EXPANSION_COEFFICIENT", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)

# Print fluid properties
print('The specific mass is: '+str(round(rho ,5))+' kg/m^3')
print('The thermal conditivity: '+str(round(k ,5))+' W/(m-K)')
print(f"The absolut viscosity: {mu} Pa.s")
print(f"The kinematic viscosity: {nu} m^2/s")
print('The Prandtl number is: '+str(round(Pr ,5)))
print('The specific heat is: '+str(round(cp ,5))+' J/kg-K')
print('The Vol. Expansivity is: '+str(round(beta ,7))+' 1/K')

Enter the fluid type (e.g. water, air): air
Enter the fluid temperature [°C]: 5
Enter the surface temperature [°C]: 12
Enter the fluid pressure [bar]: 1.01325
The specific mass is: 1.25391 kg/m^3
The thermal conditivity: 0.02501 W/(m-K)
The absolut viscosity: 1.7641516468268003e-05 Pa.s
The kinematic viscosity: 1.4069185443939391e-05 m^2/s
The Prandtl number is: 0.70956
The specific heat is: 1005.8415 J/kg-K
The Vol. Expansivity is: 0.003562 1/K


In [ ]:
import math
import numpy as np

# Função para resolver f (fator de atrito) da Equação de Colebrook. Diagrama de Moody
def f(x): 
    LHS = -2*np.log10((2.51/(Re*np.sqrt(x))) + (Epsod/3.71))
    return 1/LHS**2

# Get input parameters from user
flow_type = input("Enter the flow type (external or internal): ")
surface_type = ""
if flow_type == "external":
    surface_type = input("Enter the surface type fp* (*=flat plate) or cfc** (**=cross flow cylinder): ")
velocity = float(input("Enter the fluid velocity [m/s]: "))
diameter = 0
length = 0
if flow_type == "internal":
    diameter = float(input("Enter the tube diameter [m]: "))
    length = float(input("Enter the tube length [m]: "))
else:
    diameter = float(input("Enter the tube diameter [m]: "))
    length = float(input("Enter the surface length [m]: "))

# Calculate Reynolds number and flow regime
if flow_type == "internal":
    Re = (velocity * diameter) / nu
    if Re < 2300:
        flow_regime = "laminar"
        f = 64 / Re
        Nu = 4.36 # constant heat flux on the surface, Nu =4.36
        h = Nu * k/ diameter
    else:
        flow_regime = "turbulent"
        Eps = float(input("Enter the material roughness (Eps): [m]"))
        Epsod = Eps/diameter 
        f = f(0.001)
        #f = (-2.0*math.log10(Epsod/3.7 + 2.51/(Re*f**0.5)))**(-2) # Eq. de Colebrook. Diagrama de Moody
        #f = 0.25 / ((math.log10((diameter / 2) / (0.00015 * (3.7 * diameter / 2)**1.11)))**2)
        H_C = input("Is the fluid heating or cooling?: ")
        if H_C == "heating": 
          Nu = 0.023*Re**0.8*Pr**0.4 # Dittus-Boelter correlation
          h = Nu * k/ diameter
        else: #cooling
          Nu = 0.023*Re**0.8*Pr**0.3 # Dittus-Boelter correlation
          h = Nu * k/ diameter 
else: #external
    if surface_type == "fp":
      Re = (velocity * length) / nu
      if Re < 5e5:
        flow_regime = "laminar"
        Nu = 0.664*Re**0.5*Pr**(1/3)
        h = Nu * k/ length
      else: # turbulent
        flow_regime = "turbulent"
        Nu = (0.037 * Re**0.8 - 871)*Pr**(1/3)
        h = Nu * k/ length
    else: # surface_type == "cfc"
        Re = (velocity * diameter) / nu
        Nu = 0.3 + ((0.62*(Re**(1/2))*(Pr**(1/3)))/(1+(0.4/Pr)**(2/3))**(1/4))*(1+(Re/282000)**(5/8))**(4/5) # |Churchill & Bernstein correlation. 
        #Use to Re*Pr> 0,2. As propriedades dos fluidos devem ser avaliadas na temperatura média do fuido, Tf =(T_inf+T_s)/2
        h = Nu * k/ diameter
      
# Print results
print("Fluid properties:")
print("Thermal conductivity (k): {:.4f} W/(m*K)".format(k))
print("Kinematic viscosity (nu): {:.4e} m^2/s".format(nu))
print("Prandtl number (Pr): {:.4f}".format(Pr))
print("Specific heat (cp): {:.4f} J/(kg*K)".format(cp))

print("\nReynolds number (Re): {:.2f}".format(Re))
print("Flow regime: {}".format(flow_regime))
print("Nusselt Number: {:.2f}".format(Nu))
#print("Friction factor (f): {:.4f}".format(f))
print("\nCoefficient of forced convection (h): {:.2f} W/(m^2*K)".format(h))

Enter the flow type (external or internal): external
Enter the surface type fp* (*=flat plate) or cfc** (**=cross flow cylinder): fp
Enter the fluid velocity [m/s]: 11.67
Enter the tube diameter [m]: 1
Enter the surface length [m]: 10
Fluid properties:
Thermal conductivity (k): 0.0250 W/(m*K)
Kinematic viscosity (nu): 1.4069e-05 m^2/s
Prandtl number (Pr): 0.7096
Specific heat (cp): 1005.8415 J/(kg*K)

Reynolds number (Re): 8294723.28
Flow regime: turbulent
Nusselt Number: 10536.01

Coefficient of forced convection (h): 26.35 W/(m^2*K)


In [ ]:
# Calculate the coefficient of natural convection for external flows
if flow_type == "external":
  plate_type = input("Enter the plate type (vertical or horizontal) or vertical cylinder: ")
  if plate_type == "vertical":
        Gr = (9.81 * beta* (abs(T_s - T_inf)) * (length**3)) / (nu**2)
        Ra = Gr * Pr
        if Ra < 1e4:
            h_nat = 0
        else: 
            Nu= (0.825 + (0.387*Ra**(1/6))/(1+(0.492/Pr)**(9/16)**(8/27)))**2
            h_nat = Nu*k/ length
  if plate_type == "horizontal":
      w=1
      L_c= w*length/(2*(w+length))
      Gr = (9.81 * beta* (abs(T_s - T_inf)) * (L_c**3)) / (nu**2)
      Ra = Gr * Pr
      if Ra >= 1e4 and Ra <= 1e7:
        Nu= 0.54*Ra**(1/4)
        h_nat = Nu*k/ L_c
      if Ra >= 1e7 and Ra <= 1e11:
        Nu= 0.15*Ra**(1/3)
        h_nat = Nu*k/ L_c             
  else: # vertical cylinder
      diameter >= 35*length/Gr**(1/4)
      Nu= (0.825 + (0.387*Ra**(1/6))/(1+(0.492/Pr)**(9/16)**(8/27)))**2
      h_nat = Nu*k/ length


if flow_type == "external":
    print("Grashof number (Gr): {:.4e}".format(Gr))
    print("Rayleigh number (Ra): {:.4e}".format(Ra))
    print("Coefficient of natural convection (h_nat): {:.4f} W/(m^2*K)".format(h_nat))


Enter the plate type (vertical or horizontal) or vertical cylinder: horizontal 
Grashof number (Gr): 1.6120e+11
Rayleigh number (Ra): 1.1297e+12
Coefficient of natural convection (h_nat): 797.5463 W/(m^2*K)
